In [ ]:
import pandas as pd
import pandas_utils as pu

import matplotlib.pyplot as plt
import matplotlib
import upsetplot as ups
import seaborn as sns

In [ ]:
library = "all"
def save_fig(file_name):
  plt.rcParams["pdf.fonttype"] = 42
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.png".format(library, file_name), dpi=300, bbox_inches="tight", transparent=True)
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.pdf".format(library, file_name), bbox_inches="tight", transparent=True)
  plt.savefig(r"C:\git\msn_library\figures/{}/{}.svg".format(library, file_name), bbox_inches="tight", transparent=True)
sns.set_theme(font_scale=1, style="white")

## Overlap between public databases

In [ ]:
libraries = {
  "msnlib": r"C:\git\msn_library\data\acquisition_results\20240527_2batch_all_lib_only_detected.tsv",
  "massbankeu":  r"C:\git\msn_library\data\public_library\20231208\MassBank_nist.tsv",
  "mona": r"C:\git\msn_library\data\public_library\20231208\MoNA_LC_MSMS.tsv",
  "gnps": r"C:\git\msn_library\data\gnpslib\20240229_ALL_GNPS_NO_PROPOGATED_standardized.tsv",
  "mzcloud": r"C:\git\msn_library\data\public_library\mzCloud_04.03.2024_for Customer_standardized.tsv",
  # "nist20": r"C:\git\msn_library\data\public_library\NIST20_standardized.tsv",
  "nist23": r"C:\git\msn_library\data\public_library\NIST23tandemLib_statistics_standardized.csv",
  # "coconut2": r"C:\git\msn_library\data\compound_databases\coconut\coconut-10-2024_standardized.parquet"
}

# msnlib = [
#   "mce_bioactive",
#   "nih_natural_product",
#   "mce_scaffold_library",
#   "iocb_peptide",
# ]

open = [
  "massbankeu",
  "mona",
  "gnps",
]

open_msnlib = [
  "massbankeu",
  "mona",
  "gnps",
  "msnlib"
]

commercial = [
  "mzcloud",
  # "nist20",
  "nist23"
]


In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["split_inchikey"]].copy()
  df = df.dropna(subset="split_inchikey").drop_duplicates(["split_inchikey"]).set_index(["split_inchikey"])
  df[key] = True
  # if key in msnlib:
  #   df["msnlib"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
# merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]
# merged_df["msnlib"] = merged_df[msnlib].any(axis=1)
merged_df["open"] = merged_df[open].any(axis=1)
merged_df["commercial"] = merged_df[commercial].any(axis=1)
merged_df["available"] = merged_df[open].any(axis=1) | merged_df[commercial].any(axis=1)


merged_df

In [ ]:
save = merged_df.reset_index()
save

In [ ]:
pu.save_dataframe(save, r"C:\git\msn_library\data\acquisition_results\20240527_public_spectral_libraries_and_new_nist23.tsv")

In [ ]:
merged_df[(merged_df["msnlib"] == True)]

In [ ]:
merged_df[merged_df["nist23"] == True]

In [ ]:
merged_df[(merged_df["msnlib"] == True) & (merged_df["entries"] == 1)]

In [ ]:
subset = merged_df[(merged_df["open"] == True) & (merged_df["commercial"] != True)]
subset[(subset["open"] == True) & (subset["msnlib"] != True)]

In [ ]:
merged_df[(merged_df["msnlib"] == True) &  (merged_df["gnps"] == True)]

In [ ]:
merged_df[(merged_df["msnlib"] == True) &  (merged_df["commercial"] == False)]

In [ ]:
merged_df[(merged_df["msnlib"] == True) &  (merged_df["available"] == False)]

In [ ]:
merged_df[merged_df["gnps"] == True]

In [ ]:
msnlib_df = merged_df[(merged_df["msnlib"] == True)]
msnlib_df

In [ ]:
file_name = "20240730_msnlib_combined_public_separated"
msnlib_combined = ups.from_indicators(["msnlib", "massbankeu", "mona", "gnps", "nist23", "mzcloud"], data=merged_df)
ups.plot(msnlib_combined)
save_fig(file_name)
matplotlib.pyplot.show()

In [ ]:
file_name = "20240730_msnlib_combined_open_combined"
msnlib_combined = ups.from_indicators(["msnlib", "open", "nist23", "mzcloud"], data=merged_df)
ups.plot(msnlib_combined)
save_fig(file_name)
matplotlib.pyplot.show()

In [ ]:
open = merged_df[merged_df["open"] == True]
file_name = "20240527_open_libraries_upsetplot"
msnlib_combined = ups.from_indicators(["massbankeu", "mona", "gnps"], data=open)
ups.plot(msnlib_combined)
save_fig(file_name)

In [ ]:
open

In [ ]:
new_open = merged_df[(merged_df["open"] == True) | (merged_df["msnlib"] == True)]
file_name = "new_and_open_libraries_upsetplot"
msnlib_combined = ups.from_indicators(["msnlib", "massbankeu", "mona", "gnps"], data=new_open)
ups.plot(msnlib_combined)
save_fig(file_name)


## using smiles

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["canonical_smiles"]].copy()
  df = df.dropna(subset="canonical_smiles").drop_duplicates(["canonical_smiles"]).set_index(["canonical_smiles"])
  df[key] = True
  # if key in msnlib:
  #   df["msnlib"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
# merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]
# merged_df["msnlib"] = merged_df[msnlib].any(axis=1)
merged_df["open"] = merged_df[open].any(axis=1)
merged_df["open_msnlib"] = merged_df[open_msnlib].any(axis=1)
merged_df["commercial"] = merged_df[commercial].any(axis=1)
merged_df["available"] = merged_df[open].any(axis=1) | merged_df[commercial].any(axis=1)


merged_df

In [ ]:
merged_df[merged_df["gnps"]==True]

In [ ]:
save = merged_df.reset_index()
save

In [ ]:
pu.save_dataframe(save, r"C:\git\msn_library\data\acquisition_results\20240527_public_spectral_libraries_msnlib_nist23_coconut_smiles.tsv" )

In [ ]:
file_name = "msnlib_combined_open_combined_canonical_smiles"
msnlib_combined = ups.from_indicators(["msnlib", "open", "nist23", "mzcloud", "coconut2"], data=merged_df)
ups.plot(msnlib_combined)
# save_fig(file_name)
matplotlib.pyplot.show()

## Test with canonical SMILES

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["canonical_smiles", "split_inchikey"]].copy()
  df = df.dropna(subset="canonical_smiles").drop_duplicates(["canonical_smiles"]).set_index(["canonical_smiles", "split_inchikey"])
  df[key] = True
  # if key in msnlib:
  #   df["msnlib"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
# merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]
# merged_df["msnlib"] = merged_df[msnlib].any(axis=1)
merged_df["open"] = merged_df[open].any(axis=1)
merged_df["commercial"] = merged_df[commercial].any(axis=1)
merged_df["available"] = merged_df[open].any(axis=1) | merged_df[commercial].any(axis=1)


merged_df

In [ ]:
test = merged_df.reset_index()
test[test.duplicated(["split_inchikey"], keep=False)]